In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk import pos_tag, map_tag
from nltk.corpus import sentiwordnet as swn
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from textblob import TextBlob
from afinn import Afinn

In [2]:
df=pd.read_csv('health_news.csv')

In [3]:
df.head()

,Unnamed: 0,numbers,time,tweets
0,0,585978391360221184,Thu Apr 09 01:31:50 +0000 2015,Breast cancer risk test devised http://bbc.in/...
1,1,585947808772960257,Wed Apr 08 23:30:18 +0000 2015,GP workload harming care - BMA poll http://bbc...
2,2,585947807816650752,Wed Apr 08 23:30:18 +0000 2015,Short people's 'heart risk greater' http://bbc...
3,3,585866060991078401,Wed Apr 08 18:05:28 +0000 2015,New approach against HIV 'promising' http://bb...
4,4,585794106170839041,Wed Apr 08 13:19:33 +0000 2015,Coalition 'undermined NHS' - doctors http://bb...


In [4]:
df.drop(['numbers', 'time'], axis=1, inplace=True)

In [5]:
df.shape

(55230, 2)

In [6]:
df.isnull().values.any()

False

In [7]:
# import re
# import string
from Scripts.preprocessor import TwitterPreprocessor

In [8]:
df['clean_text'] = df.tweets.apply(lambda x: TwitterPreprocessor(x).ml_preprocess().text)

In [9]:
df.head()

,Unnamed: 0,tweets,clean_text
0,0,Breast cancer risk test devised http://bbc.in/...,breast cancer risk test devised
1,1,GP workload harming care - BMA poll http://bbc...,gp workload harming care bma poll
2,2,Short people's 'heart risk greater' http://bbc...,short people heart risk greater
3,3,New approach against HIV 'promising' http://bb...,new approach hiv promising
4,4,Coalition 'undermined NHS' - doctors http://bb...,coalition undermined nhs doctors


In [10]:
import boto3

In [27]:
df1=  df[:5000]
df1.shape

(5000, 3)

In [20]:
def create_sentiment_aws(row):
    """Uses AWS Comprehend to Create Sentiments on a DataFrame"""

    try:
      comprehend = boto3.client(service_name='comprehend', region_name="us-east-2")
      payload = comprehend.detect_sentiment(Text=row, LanguageCode='en')  
      sentiment = payload['Sentiment']
    except Exception:
      print("Size exceeded:  Fail")
      return None
    return sentiment

In [28]:
def apply_sentiment_aws(df, column="clean_text"):
    """Uses Pandas Apply to Create Sentiment Analysis"""

    df['Sentiment'] = df[column].apply(create_sentiment_aws)
    return df

In [29]:
df1_aws = apply_sentiment_aws(df1)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [33]:
df1_aws.head()

,Unnamed: 0,tweets,clean_text,Sentiment
0,0,Breast cancer risk test devised http://bbc.in/...,breast cancer risk test devised,NEUTRAL
1,1,GP workload harming care - BMA poll http://bbc...,gp workload harming care bma poll,NEUTRAL
2,2,Short people's 'heart risk greater' http://bbc...,short people heart risk greater,MIXED
3,3,New approach against HIV 'promising' http://bb...,new approach hiv promising,POSITIVE
4,4,Coalition 'undermined NHS' - doctors http://bb...,coalition undermined nhs doctors,NEUTRAL


In [32]:
df1_aws.to_csv('sentiment_for_healthtweets.csv', index='id')

In [34]:
sent_df = pd.read_csv('sentiment_for_healthtweets.csv')

In [35]:
sent_df.head()

,Unnamed: 0,Unnamed: 0.1,tweets,clean_text,Sentiment
0,0,0,Breast cancer risk test devised http://bbc.in/...,breast cancer risk test devised,NEUTRAL
1,1,1,GP workload harming care - BMA poll http://bbc...,gp workload harming care bma poll,NEUTRAL
2,2,2,Short people's 'heart risk greater' http://bbc...,short people heart risk greater,MIXED
3,3,3,New approach against HIV 'promising' http://bb...,new approach hiv promising,POSITIVE
4,4,4,Coalition 'undermined NHS' - doctors http://bb...,coalition undermined nhs doctors,NEUTRAL


In [37]:
sent_df.index = sent_df.index.rename('id')
sent_df.head()

,Unnamed: 0,Unnamed: 0.1,tweets,clean_text,Sentiment
id,,,,,
0,0,0,Breast cancer risk test devised http://bbc.in/...,breast cancer risk test devised,NEUTRAL
1,1,1,GP workload harming care - BMA poll http://bbc...,gp workload harming care bma poll,NEUTRAL
2,2,2,Short people's 'heart risk greater' http://bbc...,short people heart risk greater,MIXED
3,3,3,New approach against HIV 'promising' http://bb...,new approach hiv promising,POSITIVE
4,4,4,Coalition 'undermined NHS' - doctors http://bb...,coalition undermined nhs doctors,NEUTRAL


In [38]:
sent_df =sent_df[["clean_text", "Sentiment"]]
sent_df.head()

,clean_text,Sentiment
id,,
0,breast cancer risk test devised,NEUTRAL
1,gp workload harming care bma poll,NEUTRAL
2,short people heart risk greater,MIXED
3,new approach hiv promising,POSITIVE
4,coalition undermined nhs doctors,NEUTRAL
